In [9]:
!pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib gspread pandas


In [10]:
from googleapiclient.discovery import build
from google.oauth2.service_account import Credentials
import pandas as pd
import gspread
from datetime import datetime, timedelta


In [11]:
# Configuration
SCOPES = ['https://www.googleapis.com/auth/webmasters.readonly', 'https://www.googleapis.com/auth/spreadsheets']
SERVICE_ACCOUNT_FILE = r'C:\Users\avine\Downloads\Project\aristok-automation-441704-40507951b10b.json'  # Replace with your service account file
SEARCH_CONSOLE_SITE_URL = 'https://www.medikabazaar.com/'  # Replace with your site URL
SPREADSHEET_ID = '1G_xGUUl0yGbtkdza9EKD94dcc3LTKO2ADEOY6tdKwME'  # Replace with your Google Sheet ID


In [12]:
# Authenticate and initialize APIs
credentials = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
search_console_service = build('webmasters', 'v3', credentials=credentials)
sheets_client = gspread.authorize(credentials)


In [13]:
def fetch_search_console_data(start_date, end_date):
    request = {
        'startDate': '2024-10-01',
        'endDate': '2024-10-20',
        'dimensions': ['date'],
    }
    try:
        response = search_console_service.searchanalytics().query(siteUrl=SEARCH_CONSOLE_SITE_URL, body=request).execute()
        print("API Response:", response)  # Debugging
        data = response.get('rows', [])
        if not data:
            print("No data found for the given date range.")
            return pd.DataFrame(columns=['Date', 'Clicks', 'Impressions', 'CTR', 'Position'])
        
        return pd.DataFrame([
            {
                'Date': row['keys'][0],
                'Clicks': row.get('clicks', 0),
                'Impressions': row.get('impressions', 0),
                'CTR': row.get('ctr', 0),
                'Position': row.get('position', 0)
            }
            for row in data
        ])
    except Exception as e:
        print(f"Error fetching data: {e}")
        return pd.DataFrame(columns=['Date', 'Clicks', 'Impressions', 'CTR', 'Position'])



In [14]:
def calculate_metrics(data):
    data['Date'] = pd.to_datetime(data['Date'])
    data.sort_values('Date', inplace=True)
    
    # Calculate previous day values and percentage changes
    data['Previous Clicks'] = data['Clicks'].shift(1)
    data['Clicks % Change'] = ((data['Clicks'] - data['Previous Clicks']) / data['Previous Clicks']) * 100

    data['Previous Impressions'] = data['Impressions'].shift(1)
    data['Impressions % Change'] = ((data['Impressions'] - data['Previous Impressions']) / data['Previous Impressions']) * 100

    data['Previous CTR'] = data['CTR'].shift(1)
    data['CTR % Change'] = ((data['CTR'] - data['Previous CTR']) / data['Previous CTR']) * 100

    data['Previous Position'] = data['Position'].shift(1)
    data['Position % Change'] = ((data['Position'] - data['Previous Position']) / data['Previous Position']) * 100

    return data


In [15]:
def export_to_google_sheets(sheet_id, data):
    try:
        # Replace NaN values with appropriate defaults (e.g., 0 or an empty string)
        clean_data = data.fillna('')  # Replace NaN with an empty string

        # Attempt to open the sheet
        print(f"Attempting to access Google Sheet with ID: {sheet_id}")
        sheet = sheets_client.open_by_key(sheet_id)
        worksheet = sheet.sheet1

        # Clear existing content and update with cleaned data
        worksheet.clear()
        worksheet.update([clean_data.columns.values.tolist()] + clean_data.values.tolist())
        print("Data successfully exported to Google Sheets.")
    except gspread.exceptions.SpreadsheetNotFound:
        print(f"Spreadsheet with ID {sheet_id} not found.")
    except PermissionError:
        print("Permission denied: Ensure the service account email has been granted Editor access to the Google Sheet.")
    except Exception as e:
        print(f"An error occurred: {e}")


In [16]:
if __name__ == '__main__':
    # Define date ranges
    today = datetime.now().date()
    yesterday = today - timedelta(days=1)
    two_days_ago = today - timedelta(days=2)

    # Fetch data for the last 3 days
    df = pd.concat([
        fetch_search_console_data(two_days_ago.isoformat(), yesterday.isoformat()),
        fetch_search_console_data(yesterday.isoformat(), today.isoformat())
    ])

    # Calculate metrics
    metrics = calculate_metrics(df)

    # Reformat for Google Sheets
    metrics['Date Comparison'] = metrics['Date'].dt.strftime('%d %b\'%y') + ' vs ' + metrics['Date'].shift(1).dt.strftime('%d %b\'%y')
    sheet_data = metrics[['Date Comparison', 'Clicks', 'Previous Clicks', 'Clicks % Change', 
                          'Impressions', 'Previous Impressions', 'Impressions % Change',
                          'CTR', 'Previous CTR', 'CTR % Change',
                          'Position', 'Previous Position', 'Position % Change']]

    # Export to Google Sheets
    export_to_google_sheets(SPREADSHEET_ID, sheet_data)


API Response: {'rows': [{'keys': ['2024-10-01'], 'clicks': 1213, 'impressions': 141857, 'ctr': 0.008550864603086206, 'position': 14.060835912221462}, {'keys': ['2024-10-02'], 'clicks': 1104, 'impressions': 139604, 'ctr': 0.007908082862955216, 'position': 13.633033437437323}, {'keys': ['2024-10-03'], 'clicks': 1335, 'impressions': 154693, 'ctr': 0.008629996185994195, 'position': 13.279320977678369}, {'keys': ['2024-10-04'], 'clicks': 1210, 'impressions': 154994, 'ctr': 0.007806753809824897, 'position': 13.197388285998167}, {'keys': ['2024-10-05'], 'clicks': 1029, 'impressions': 136345, 'ctr': 0.007547031427628442, 'position': 14.021034874766219}, {'keys': ['2024-10-06'], 'clicks': 969, 'impressions': 135733, 'ctr': 0.007139015567327032, 'position': 13.576970965056397}, {'keys': ['2024-10-07'], 'clicks': 1284, 'impressions': 167104, 'ctr': 0.007683837610111068, 'position': 13.260795672156261}, {'keys': ['2024-10-08'], 'clicks': 1224, 'impressions': 163006, 'ctr': 0.007508926051801774, 'p